# Code to test QuasiCopula.jl with linear mixed model framework

+ We consider the longitudinal setting. In LMM, 

$$\mathbf{y}_i = \mathbf{X}_i\mathbf{\beta} + \mathbf{Z}_i\mathbf{\gamma} + \epsilon_i$$

+ The fitting code is adapted from: https://github.com/Hua-Zhou/GLMCopula.jl
+ Test code is adapted from: https://github.com/Hua-Zhou/GLMCopula.jl/blob/master/test/sleepstudy.jl

In [1]:
using Revise, QuasiCopula, RDatasets, Test, LinearAlgebra


# Dataframe with columns: Reaction (Float64), Days (Int32), Subject (Categorical)
# 180 rows
sleepstudy = dataset("lme4", "sleepstudy")

Precompiling QuasiCopula
  ✓ QuasiCopula
  1 dependency successfully precompiled in 5 seconds. 165 already precompiled.


Row,Reaction,Days,Subject
,Float64,Int32,Cat…
1,249.56,0,308
2,258.705,1,308
3,250.801,2,308
4,321.44,3,308
5,356.852,4,308
6,414.69,5,308
7,382.204,6,308
8,290.149,7,308
9,430.585,8,308


In [9]:
# form LMM data struct
groups = unique(sleepstudy[!, :Subject])
n, p, m = length(groups), 2, 1
gcs = Vector{GaussianCopulaLMMObs{Float64}}(undef, n)
for (i, grp) in enumerate(groups)
    gidx = sleepstudy[!, :Subject] .== grp
    ni = count(gidx)
    yi = Float64.(sleepstudy[gidx, :Reaction])
    Ui = Float64.(sleepstudy[gidx, :Days])
    Xi = [ones(ni, 1) Ui]
    Zi = Xi
#     Zi = ones(ni, 1)
    gcs[i] = GaussianCopulaLMMObs(yi, Xi, Zi)
end
gcm = GaussianCopulaLMMModel(gcs);

# initialize β and τ from least squares solution
@info "Initial point:"
init_β!(gcm)
@show gcm.β
@show gcm.τ[1]
gcm.Σ .= diagm(ones(2))
# gcm.Σ .= 1
@show gcm.Σ
@show loglikelihood!(gcm, true, false)
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ

# # initialize from LMM estimate
# @info "Initial point:"
# gcm.β .= [251.405, 10.4673]
# gcm.τ[1] = inv(654.94145)
# gcm.Σ .= [565.51069 10.87591; 10.87591 32.68212]
# @show loglikelihood!(gcm, true, false)
# @show gcm.∇β
# @show gcm.∇τ
# @show gcm.∇Σ

# fit model
@time QuasiCopula.fit!(gcm)
@show gcm.β
@show gcm.τ
@show gcm.Σ
@show loglikelihood!(gcm, true, false)
# @test loglikelihood!(gcm, true, false) ≈ -163.35545251
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ;

gcm.β = [251.40510484848488, 10.46728595959596]
gcm.τ[1] = 0.0004441684924519196
gcm.Σ = [1.0 0.0; 0.0 1.0]
loglikelihood!(gcm, true, false) = -949.1827705115784
gcm.∇β = [-0.5828404268983784, -3.6867547573696373]
gcm.∇τ = [37690.66749687556]
gcm.∇Σ = [0.726304670998114 -0.22102101114869246; -0.22102101114869246 -1.8640855881921932]
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequali

[ Info: Initial point:


gcm.β = [237.14119968194956, 10.458444443907405]
gcm.τ = [0.0004883043269460731]
gcm.Σ = [4.895921175206345e29 -3.0874648674506864e16; -3.0874648674506864e16 1947.016499370117]
loglikelihood!(gcm, true, false) = -939.86182327495
gcm.∇β = [0.0029085513301063542, 0.01949974628184392]
gcm.∇τ = [134.07580259522638]
gcm.∇Σ = [1.3701195734935899e-42 1.0887716276159565e-29; 1.0887716276159565e-29 9.837818484390647e-29]


In [4]:
# LMM estimate
# @info "Initial point:"
# gcm.β .= [251.405, 10.4673]
# gcm.τ[1] = inv(654.94145)
# gcm.Σ .= [565.51069 10.87591; 10.87591 32.68212]
# @show loglikelihood!(gcm, true, false)
# @show gcm.∇β
# @show gcm.∇τ
# @show gcm.∇Σ

In [ ]:
# initial logl
-949.1827705115784

# final logl
-939.86182327495